<a href="https://colab.research.google.com/github/sfekih/2020/blob/master/projet_final_iapr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')
PROJECT_PATH = os.path.join('drive', 'My Drive', 'Colab Notebooks', 'iapr', 'train_games')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.chdir(PROJECT_PATH)
!pwd

/content/drive/My Drive/Colab Notebooks/iapr/train_games


In [4]:
ls

game1/  game2/  game3/  game4/  game5/  game6/  game7/


In [5]:
#cd ..

In [6]:
import pandas as pd
import numpy as np

from PIL import Image
from scipy.spatial import distance_matrix


In [7]:
import os

In [9]:
def separate_columns(row, game_nb) :
  row['P1_suit'] = row.P1[1]
  row['P1_digit'] = row.P1[0]
  row['P2_suit'] = row.P2[1]
  row['P2_digit'] = row.P2[0]
  row['P3_suit'] = row.P3[1]
  row['P3_digit'] = row.P3[0]
  row['P4_digit'] = row.P4[1]
  row['P4_suit'] = row.P4[0]
  row['game_nb'] = game_nb
  return row

In [10]:
import cv2
import matplotlib.pyplot as plt


In [11]:
#read as BGR


In [12]:
def get_green_contours (img) :
  ## convert to hsv
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

  ## mask of green (36,25,25) ~ (86, 255,255)
  # mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,255))
  mask = cv2.inRange(hsv, (36, 25, 25), (70, 255,255))

  ## slice the green
  imask = mask>0
  green = np.zeros_like(img, np.uint8)
  green[imask] = 255
  greyscale = cv2.cvtColor(green, cv2.COLOR_BGR2GRAY)
  return greyscale


In [13]:
def find_suits (contours_card) :
  centers = np.zeros((len(contours_card), 2))
  for i in range (0, len(contours_card)):
    M = cv2.moments(contours_card[i])
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    centers[i, 0] = cX
    centers[i, 1] = cY

  distances = distance_matrix(centers, centers)
  (suit1, val), suit2 = max(map(lambda x: (max(enumerate(x[1]), key= lambda x: x[1]), x[0]),
                                enumerate(distances)),
                              key=lambda x: x[0][1])

  return suit1, suit2

def get_contours_one_card (img, ordered_contours_cards, num_player):
  cimg_copy = np.zeros((img.shape[0], img.shape[1]))

  one_card = cv2.drawContours(cimg_copy.copy(), ordered_contours_cards.copy(), num_player, color=255, thickness=-1)

  mask_cards_only = one_card < 1
  img_cards_only = img.copy()
  img_cards_only[mask_cards_only, : ] = 0

  greyscale_one_card = cv2.cvtColor(img_cards_only.copy(), cv2.COLOR_BGR2GRAY)

  _,img_cards_only_copy = cv2.threshold(greyscale_one_card.copy(),170,255,cv2.THRESH_BINARY)

  img_cards_only_copy = np.rot90(img_cards_only_copy, k=num_player)

  contours_card, _ = cv2.findContours(img_cards_only_copy, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)


  contours_card = [contour_ for contour_ in contours_card if cv2.contourArea(contour_)>500]
  contours_card = sorted(contours_card, key=lambda x: cv2.contourArea(x))[::-1][1:]

  return contours_card


def get_needed_contours (contours_card, used_for:str, suit1, suit2):

  if used_for=='digits':
    final_contours = [contours_card[i] for i in range (0, len(contours_card)) if (i!=suit1 and i!=suit2)]
  elif used_for=='suits':
    final_contours = [contours_card[j] for j in range (0, len(contours_card)) if (j==suit1 or j==suit2)]
  else :
    raise NameError ("parameter used_for should be 'digits' or 'suits'")

  
  min_x = np.min(final_contours[0][:,0,0])
  min_y = np.min(final_contours[0][:,0,1])

  for i in range (len(final_contours)) :
    final_contours[i][:,0,0] = final_contours[i][:,0,0] - min_x + 1
    final_contours[i][:,0,1] = final_contours[i][:,0,1] - min_y + 1
  if used_for=='digits':
    img_cards_only_copy = np.zeros((350, 350))
  else :
    img_cards_only_copy = np.zeros((150, 150))

  drawn_contour = cv2.drawContours(img_cards_only_copy, final_contours, -1, color=255, thickness=-1)
  return drawn_contour

In [14]:
digits = []
suits = []

In [16]:
games_paths = sorted(['game'+str(i) for i in range (1, 8)])
rounds_paths = sorted([str(i)+'.jpg' for i in range (1, 14)])
total_results = pd.DataFrame()

for game_nb in range (0, len(games_paths)) :
  
  results_path = games_paths[game_nb] + '/' + 'game'+str(game_nb + 1)+'.csv'



  data_game_1 = pd.read_csv(results_path)
  data_game_preprocessed = data_game_1.apply(lambda x: separate_columns(x, game_nb+1), axis=1)\
                                      .drop(columns={'P1', 'P2', 'P3', 'P4'})\
                                      .rename(columns={'Unnamed: 0':'round_nb'})

  total_results = total_results.append(data_game_preprocessed)

  for round_nb in range (len(rounds_paths)) :
    round = rounds_paths[round_nb]
    image_path = games_paths[game_nb] + '/' + round
    img = cv2.imread(image_path)

    try :
      green_contours = get_green_contours(img)
      contours, _ = cv2.findContours(green_contours, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x))[::-1][:5]

      cimg = np.zeros_like(green_contours)
      cimg_copy = cimg
      centers = np.zeros((5, 2))
      for i in range (5):    
        M = cv2.moments(cntsSorted[i])
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centers[i, 0] = cX
        centers[i, 1] = cY

      centers_cards = centers[:-1,:]
      center_badge = centers[-1,:]
      player1 = np.argmax(centers_cards[:,1])
      player3 = np.argmin(centers_cards[:,1])
      player2 = np.argmin(centers_cards[:,0])
      player4 = np.argmax(centers_cards[:,0])
      players_args = [player1, player2, player3, player4]
      centers_ordered = np.array([centers_cards[i,:] for i in players_args])
      ordered_contours_cards = [cntsSorted[i] for i in players_args]

      cards_only = cv2.drawContours(cimg.copy(), ordered_contours_cards.copy(), -1, color=255, thickness=-1)
      cards_and_badge = cv2.drawContours(cimg.copy(), ordered_contours_cards.copy(), -1, color=255, thickness=-1)

      cimg_2 = cv2.drawContours(cimg.copy(), cntsSorted, -1, color=255, thickness=-1)

      centers_distances = np.array([np.linalg.norm(centers_ordered[i, :]-center_badge) for i in range (4)])
      badge_idx = np.argmin(centers_distances)


      for iter in range (0, 4) :
        
        contours_card = get_contours_one_card (img, ordered_contours_cards, iter)

        suit1, suit2 = find_suits (contours_card)

        digits.append(get_needed_contours (contours_card, 'digits', suit1, suit2))
        suits.append(get_needed_contours (contours_card, 'suits', suit1, suit2))
    except Exception:
      print(image_path)

game1/10.jpg
game1/11.jpg
game1/12.jpg
game1/2.jpg
game1/3.jpg
game1/4.jpg
game1/5.jpg
game1/6.jpg
game1/7.jpg
game1/9.jpg
game2/1.jpg
game2/10.jpg


KeyboardInterrupt: ignored

In [ ]:
assert(1==2)

In [ ]:
#cntsSorted[3]
#players_args

In [ ]:
#ordered_contours_cards[0]

In [ ]:

#print ('player who has badge is :', badge_idx + 1)

In [ ]:
plt.imshow(suits[1])

In [ ]:
plt.imshow(digits[1])

In [ ]:

  """#sign_contour = contours_card[0]
  final_contours = [contours_card[i] for i in range (0, len(contours_card)) if (i!=suit1 and i!=suit2)]
  min_x = np.min(final_contours[0][:,0,0])
  min_y = np.min(final_contours[0][:,0,1])

  for i in range (len(final_contours)) :
    final_contours[i][:,0,0] = final_contours[i][:,0,0] - min_x + 1
    final_contours[i][:,0,1] = final_contours[i][:,0,1] - min_y + 1

  img_cards_only_copy = np.zeros((350, 350))
  digit = cv2.drawContours(img_cards_only_copy, final_contours, -1, color=255, thickness=-1)"""
  



  """suits_contours = [contours_card[j] for j in range (0, len(contours_card)) if (j==suit1 or j==suit2)]
  idx = np.argmax([suits_contours[0][:, 0, 1].max(), suits_contours[1][:, 0, 1].max()])
  color_contour = suits_contours[idx]
  min_x = np.min(color_contour[:,0,0])
  min_y = np.min(color_contour[:,0,1])

  color_contour[:,0,0] = color_contour[:,0,0] - min_x + 1
  color_contour[:,0,1] = color_contour[:,0,1] - min_y + 1

  img_cards_only_copy = np.zeros((150, 150))
  cards_only = cv2.drawContours(img_cards_only_copy, [color_contour], -1, color=255, thickness=-1)"""

In [ ]:
#rotate images depending on player
#put in functions
#iterate